# Hadron charges / form factors

### Parameter list

In [ ]:
# Commonly changed parameters
const hadron = "rho"
const γ = 8
const p_src = (-1,0,0)
const p_snk = (0,0,0)
const fit_data = false

# Other parameters
const size_s = 16
const size_t = 60
const m = "-8999"
const q = (p_snk[1]-p_src[1],p_snk[2]-p_src[2],p_snk[3]-p_src[3])
const sources = ["DG0_1", "DG1_1", "DG1_1", "DG2_1", "DG2_1"]
const sinks = ["DG0_1", "DG1_1", "DG1_1", "DG2_1", "DG2_1"]
const file_prefix = "/home/arios/Documents/LQCDConfigs/wil_16_60_aniso_cluster/" * hadron * "/"

if γ ∈ [1,2,4,8,7,11,13,14]
    const curr = "nonlocal"
else
    const curr = "local"
end

if hadron == "rho"
    const t_gen_ev = 0
    const t_var = 10
    const t_sn = 10
    const t_sink = 21
    const γ_ins = 1
    const had = "rho_x"
    const seqsource = "a0-rho_x_1";
elseif hadron == "nucleon_pol"
    const t_gen_ev = 0
    const t_var = 10
    const t_sn = 10
    const t_sink = 21
    const γ_ins = 1
    const had = "rho_x"
    const seqsource = "a0-rho_x_1";
elseif hadron == "nucleon_unpol"
    const t_gen_ev = 0
    const t_var = 10
    const t_sn = 10
    const t_sink = 21
    const γ_ins = 1
    const had = "rho_x"
    const seqsource = "a0-rho_x_1";
end

### Paremeter description

- size_s: spatial size.
- size_t: temporal size.
- t_gen_ev: time-slice used for generalized eigenvalue problem.
- t_var: time-slice used for picking the variational vector.
- t_sn: time-slice used for picking the s/n optimized vector.
- t_sink: time of the sink.
- g: gamma matrix of interest (see table below).
- i: gamma insertion at the source.
- file_prefix: location of the data.
- sources/sinks: vectors containning types of sources/sinks.

### $\gamma$-matrices
\begin{array}{|ccc|ccc|} \hline
\Gamma & n_\Gamma & \text{state} & \Gamma & n_\Gamma & \text{state} \\ \hline
1 & 0 & a_0 & \gamma_4 & 8 & a_0 \\ \hline
\gamma_1 & 1 & \rho(x) & \gamma_1\gamma_4 & 9 & \rho(x) \\ \hline
\gamma_2 & 2 & \rho(y) & \gamma_2\gamma_4 & 10 & \rho(y) \\ \hline
\gamma_1\gamma_2 & 3 & b_1(z) & \gamma_3\gamma_5 & 11 & a_1(z) \\ \hline
\gamma_3 & 4 & \rho(z) & \gamma_3\gamma_4 & 12 & \rho(z) \\ \hline
\gamma_1\gamma_3 & 5 & -b_1(y) & -\gamma_2\gamma_5 & 13 & -a_1(y) \\ \hline
\gamma_2\gamma_3 & 6 & b_1(x) & \gamma_1\gamma_5 & 14 & a_1(x) \\ \hline
-\gamma_4\gamma_5 & 7 & \pi & \gamma_5 & 15 & \pi \\ \hline
\end{array}


### Load modules and data

In [ ]:
push!(LOAD_PATH, pwd())
using lqcd, lqcdfits, PyPlot

const had_3ptfn = read_bar3ptfn_file(seqsource, curr, γ_ins, γ, q, p_snk, sources, sinks, file_prefix)
const had_2ptfn_src = read_hadspec_file(had, m, p_src, sources, sinks, file_prefix)
const had_2ptfn_snk = read_hadspec_file(had, m, p_snk, sources, sinks, file_prefix);

### Process data

In [ ]:
had_ff, had_ff_err, had_allff = find_ff_with2ptfn(had_3ptfn, had_2ptfn_src, had_2ptfn_snk, t_sink, t_gen_ev, t_var, t_sn);

### Plots using smeared sources and sinks

In [ ]:
type_labels = ["Point", "\$G1\$", "\$\\nabla^2 G1\$", "\$G2\$", "\$\\nabla^2 G2\$", "Var", "Var + S/N", "S/N"]
figure(1, figsize=(16, 16))
for x in 1:8
    subplot(810+x)
    ax = gca()
    errorbar(0:(length(had_ff[x,:])-1), had_ff[x,:], yerr=had_ff_err[x,:], color="b", ecolor="b", capsize=2, fmt="o")
    ylabel("\$R\$", fontsize=20)
    text(10., 0.88, type_labels[x], fontsize=20)
    xlim(-0.5, t_sink+0.5)
    ylim(0.4, 1)
    yticks(0.4:0.2:1.)
    if x == 5
        xlabel("\$\\tau\$", fontsize=20)
    else
        xticks([])
    end
    for tick in ax[:yaxis][:get_major_ticks]()
            tick[:label][:set_fontsize](16)
    end
    if x == 0
        plt.suptitle(hadron, fontsize=30)
    end
end

### Plots using variational method and s/n optimization

In [ ]:
type_labels = ["Point", "\$G1\$", "\$\\nabla^2 G1\$", "\$G2\$", "\$\\nabla^2 G2\$", "Var", "Var + S/N", "S/N"]
figure(1, figsize=(16, Int64(floor(12/5*3))))
for x in 1:3
    subplot(310+x)
    errorbar(0:(length(had_ff[x+5,:])-1), had_ff[x+5,:], yerr=had_ff_err[x+5,:], color="b", ecolor="b", capsize=2, fmt="o")
    ylabel("\$R\$", fontsize=20)
    text(10., 0.9, type_labels[x+5], fontsize=20)
    xlim(-0.5, t_sink+0.5)
    ylim(0.4, 1)
    if x == 3
        xlabel("\$\\tau\$", fontsize=20)
    end
end

In [ ]:
# testing fitting
for x in 5:19
    println("x=",x)
    fit_corr(had_allff[:,4,:], 3, x, [0.4, 1.])
end